In [21]:
input_file = open("./input_9.txt", "r")

In [22]:
# artwork = [(x, y), ...]
artworks = []
# cameras = [{"price": 1, "radius": 2}, ...]
cameras = []

for i, line in enumerate(input_file.readlines()):
    line = line.strip("\n").split(",")
    line = (int(line[0]),int(line[1]))
    if i>=2:
        artworks.append(line)
    elif i==0:
        for j, rad in enumerate(line):
            cameras.append({})
            cameras[j]["radius"] = rad
    elif i==1:
        for k, price in enumerate(line):
            cameras[k]["price"] = price

print(*artworks[0:5])
print(cameras)
camera_1 = cameras[0]
camera_2 = cameras[1]
print(camera_1, camera_2)

(301, 628) (152, 506) (543, 152) (358, 17) (655, 642)
[{'radius': 4, 'price': 1}, {'radius': 8, 'price': 2}]
{'radius': 4, 'price': 1} {'radius': 8, 'price': 2}


In [23]:
from pyscipopt import Model, quicksum

In [24]:
model = Model("museum")

In [25]:
cameras_1 = []
cameras_2 = []
grid_size = max(max([artwork[0], artwork[1]] for artwork in artworks))
grid_size = 1000
print(grid_size)
for i in range(grid_size):
    cameras_1.append([])
    cameras_2.append([])
    for j in range(grid_size):
        cameras_1[i].append(model.addVar(name="cameras_1_{}_{}".format(i,j), vtype="I", lb=0, ub=2)) # NB : < ou <= ??
        cameras_2[i].append(model.addVar(name="cameras_2_{}_{}".format(i,j), vtype="I", lb=0, ub=2))

1000


In [26]:
model.setObjective(\
                   camera_1["price"]*quicksum(cameras_1[i][j] for i in range(grid_size) for j in range(grid_size))\
                   + camera_2["price"]*quicksum(cameras_2[i][j] for i in range(grid_size) for j in range(grid_size)),\
                   "minimize")

In [27]:
def distance(i, j, x, y):
    return ((i - x)**2 + (j - y)**2)**(1/2)

for artwork in artworks:
    i = artwork[0]
    j = artwork[1]
    i_min1 = i - camera_1["radius"] if i - camera_1["radius"]>0 else 0
    i_max1 = i + camera_1["radius"] if i + camera_1["radius"]<len(cameras_1) else len(cameras_1)
    j_min1 = j - camera_1["radius"] if j - camera_1["radius"]>0 else 0
    j_max1 = j + camera_1["radius"] if j + camera_1["radius"]<len(cameras_1) else len(cameras_1)
    
    i_min2 = i - camera_2["radius"] if i - camera_2["radius"]>0 else 0
    i_max2 = i + camera_2["radius"] if i + camera_2["radius"]<len(cameras_2) else len(cameras_2)
    j_min2 = j - camera_2["radius"] if j - camera_2["radius"]>0 else 0
    j_max2 = j + camera_2["radius"] if j + camera_2["radius"]<len(cameras_2) else len(cameras_2)
    
    accurate_cameras = [cameras_1[x][y] for x in range(i_min1, i_max1) for y in range(j_min1, j_max1) if distance(i, j, x, y)-camera_1["radius"]<=0] +\
    [cameras_2[x][y] for x in range(i_min2, i_max2) for y in range(j_min2, j_max2) if distance(i, j, x, y)-camera_2["radius"]<=0]

    model.addCons(quicksum(camera for camera in accurate_cameras)>=1)

In [28]:
model.optimize()

In [29]:
print(model.getObjVal())

2700.0


In [30]:
output_file = open("solution.txt", "w")

for i, line in enumerate(cameras_1):
    for j, camera_1 in enumerate(line):
        if model.getVal(camera_1) == 1:
            output_file.write("{},{},{}\n".format("1",i,j))

for i, line in enumerate(cameras_2):
    for j, camera_2 in enumerate(line):
        if model.getVal(camera_2) == 1:
            output_file.write("{},{},{}\n".format("2",i,j))
            
output_file.close()